In [1]:
import sys
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from copy import deepcopy
from tqdm import tqdm
import random
from random import shuffle

In [2]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
import collections
%matplotlib inline

def live_plot(data, figsize=(16,8), title=''):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    plt.imshow(convert_to_rgb(data.astype(int)))
    plt.title(title)
    plt.show()

def convert_to_rgb(img):
    img = deepcopy(img)
    b,g,r = cv2.split(img)  
    img = cv2.merge([r,g,b])
    return img

In [3]:
file_num = 70
frame_dir = f'../scene_data/{file_num}'
frame_bt_dir = f'../scene_data/{file_num}_back_true'

In [4]:
scene_dict = {}
for file_name in os.listdir(frame_dir):
    if file_name[0] != '.':
        num, verti_pos, hori_pos, real_back_prob = file_name.replace('.jpg', '').split('_')
        scene_dict[int(num)] = {'file_name': file_name, 'num':num, 'verti_pos':int(verti_pos), 'hori_pos':int(hori_pos)}

In [5]:
scene_bt_dict = {}
for file_name in os.listdir(frame_bt_dir):
    if file_name[0] != '.':
        num, verti_pos, hori_pos, real_back_prob = file_name.replace('.npy', '').split('_')
        scene_bt_dict[int(num)] = {'file_name': file_name, 'num':num, 'verti_pos':int(verti_pos), 'hori_pos':int(hori_pos)}

In [6]:
scene_num_0 = random.choice(sorted(list(scene_dict.keys()))[5:-5])
file_name_0 = scene_dict[scene_num_0]['file_name']
file_name_bt_0 = scene_bt_dict[scene_num_0]['file_name']
frame_0 = cv2.imread(f"{frame_dir}/{file_name_0}")
frame_bt_0 = np.load(f"{frame_bt_dir}/{file_name_bt_0}").astype(int)

n = 1

set_shape = (n*2, n*2, 3)

compare_frame_dict = {}

for i in range(-3,4):
    if i == 0:
        continue
    m_num = 3 + 2 * abs(i)
    compare_frame_dict[i] = {}
    scene_num = scene_num_0 + i
    file_name = scene_dict[scene_num]['file_name']
    file_name_bt = scene_bt_dict[scene_num]['file_name']
    
    abs_pos_0 = np.array((scene_dict[scene_num_0]['verti_pos'], scene_dict[scene_num_0]['hori_pos']))
    abs_pos_1 = np.array((scene_dict[scene_num]['verti_pos'], scene_dict[scene_num]['hori_pos']))
    
    frame_1 = cv2.imread(f"{frame_dir}/{file_name}")
    frame_bt_1 = np.load(f"{frame_bt_dir}/{file_name_bt}").astype(int)
    compare_frame_dict[i]['euclidean_dis'] = np.sqrt(np.sum(np.power(abs_pos_1 - abs_pos_0, 2)/2))
    
    compare_dict = {}
    for s0 in tqdm(range(n, frame_0.shape[0]), ncols=70):
        for s1 in range(n, frame_0.shape[1]):
            segment_0 = frame_0[s0-n:s0+n, s1-n:s1+n, :]
            if segment_0.shape == set_shape:
                temp_compare = {}
                for m1 in range(-m_num, m_num+1):
                    for m2 in range(-m_num, m_num+1):
                        compare_1 = frame_1[max(s0-n+m1,0):max(s0+n+m1,0), max(s1-n+m2,0):max(s1+n+m2,0), :]
                        if compare_1.shape == set_shape:
                            diff_array = segment_0 - compare_1
                            diff = np.sum(np.power(diff_array,2))/len(diff_array)
                            temp_compare[(m1,m2)] = diff
                temp_compare = {k: v for k, v in sorted(temp_compare.items(), key=lambda item: item[1])}
                compare_dict[(s0,s1)] = np.array(list(temp_compare.keys())[0])

    compare_mean_dict = {}
    for key in tqdm(compare_dict, ncols=70):
        compare_mean_dict[key] = []
        s0, s1 = key
        for c0 in range(s0-n,s0+n):
            for c1 in range(s1-n, s1+n):
                if (c0, c1) in compare_dict:
                    compare_mean_dict[key].append(compare_dict[(c0, c1)])
    compare_frame_dict[i]['compare_mean_dict'] = compare_mean_dict

100%|█████████████████████| 398701/398701 [00:01<00:00, 301517.09it/s]


In [7]:
np.save(f'{file_num}_hedge_back_fast.npy', compare_frame_dict)
np.save(f'{file_num}_hedge_back_frame_bt_0_fast.npy', frame_bt_0)

In [8]:
for i in compare_frame_dict:
    compare_mean_dict = compare_frame_dict[i]['compare_mean_dict']
    euclidean_dis = compare_frame_dict[i]['euclidean_dis']
    
    for key in tqdm(compare_mean_dict, ncols=70):
        compare_mean_dict[key] = np.mean(compare_mean_dict[key], axis=0)
        compare_mean_dict[key] = np.sqrt(np.sum(np.power(compare_mean_dict[key], 2)/2))

100%|██████████████████████| 398701/398701 [00:11<00:00, 35057.75it/s]


In [26]:
for i in tqdm(compare_frame_dict):
    compare_mean_dict = compare_frame_dict[i]['compare_mean_dict']
    euclidean_dis = compare_frame_dict[i]['euclidean_dis']

    back_compare_mean_dict = {k: v for k, v in sorted(compare_mean_dict.items(), key=lambda item: item[1]) if v <= euclidean_dis}
    hedge_compare_mean_dict = {k: v for k, v in sorted(compare_mean_dict.items(), key=lambda item: item[1]) if v > euclidean_dis * 1.5}
    compare_frame_dict[i]['back'] = back_compare_mean_dict
    compare_frame_dict[i]['hedge'] = hedge_compare_mean_dict

100%|██████████| 6/6 [00:07<00:00,  1.25s/it]


In [27]:
# i = -1

In [28]:
# compare_frame_dict[i]['back']

In [31]:
back_compare_mean_dict = {}
hedge_compare_mean_dict = {}
for key in compare_frame_dict:
    if len(back_compare_mean_dict) and 0:
        back_compare_mean_dict = {k:v for k, v in back_compare_mean_dict.items() if k in compare_frame_dict[key]['back']}
    else:
        back_compare_mean_dict = compare_frame_dict[key]['back']
        
    if len(hedge_compare_mean_dict) and 1:
        hedge_compare_mean_dict = {k:v for k, v in hedge_compare_mean_dict.items() if k in compare_frame_dict[key]['hedge']}
    else:
        hedge_compare_mean_dict = compare_frame_dict[key]['hedge']
    
    back_accurate_list = []
    for i, j in back_compare_mean_dict:
        if sum(frame_bt_0[i][j])==3:
            back_accurate_list.append(1)
        else:
            back_accurate_list.append(0)
#     print(key, 'back_accuracy', sum(back_accurate_list)/len(back_accurate_list), len(back_accurate_list))
    
    hedge_accurate_list = []
    for i, j in hedge_compare_mean_dict:
        if sum(frame_bt_0[i][j]) == 0:
            hedge_accurate_list.append(1)
        else:
            hedge_accurate_list.append(0)
    print(key, 'hedge_accuracy', sum(hedge_accurate_list)/len(hedge_accurate_list), len(hedge_accurate_list))

-3 hedge_accuracy 0.38036995855655514 9893
-2 hedge_accuracy 0.35191829975158706 3623
-1 hedge_accuracy 0.39509014192558495 2607
1 hedge_accuracy 0.5053763440860215 1581
2 hedge_accuracy 0.6017699115044248 904
3 hedge_accuracy 0.6688632619439868 607


In [ ]:
# i = 2

In [ ]:
# back_compare_mean_dict = compare_frame_dict[i]['back']
# hedge_compare_mean_dict = compare_frame_dict[i]['hedge']

In [ ]:
for i in range(-3,4):
    if i == 0:
        continue
    back_compare_mean_dict = compare_frame_dict[i]['back']
    back_accurate_list = []
    for i, j in back_compare_mean_dict:
        if sum(frame_bt_1[i][j])==3:
            back_accurate_list.append(1)
        else:
            back_accurate_list.append(0)
    print(sum(back_accurate_list)/len(back_accurate_list))

In [ ]:
for i in range(-3,4):
    if i == 0:
        continue
    hedge_compare_mean_dict = compare_frame_dict[i]['hedge']
    hedge_accurate_list = []
    for i, j in hedge_compare_mean_dict:
        if sum(frame_bt_1[i][j]) == 0:
            hedge_accurate_list.append(1)
        else:
            hedge_accurate_list.append(0)
    print(sum(hedge_accurate_list)/len(hedge_accurate_list))

In [ ]:
back_accurate_list = []
for i, j in back_compare_mean_dict:
    if sum(frame_bt_1[i][j])==3:
        back_accurate_list.append(1)
    else:
        back_accurate_list.append(0)
print(sum(back_accurate_list)/len(back_accurate_list))

In [ ]:
hedge_accurate_list = []
for i, j in hedge_compare_mean_dict:
    if sum(frame_bt_1[i][j]) == 0:
        hedge_accurate_list.append(1)
    else:
        hedge_accurate_list.append(0)
print(sum(hedge_accurate_list)/len(hedge_accurate_list))

In [ ]:
len(back_accurate_list), len(hedge_accurate_list)